# SpikeInterface v0.101.2 - Adapted by Rodrigo Noseda - November 2024

SpikeInterface to analyze a multichannel dataset from Cambridge Neurotech Probes. 
The dataset is extracted using open-ephys DAQ and Bonsai-rx (in .bin).
Event_timestamps need some work.

# 0. Preparation <a class="anchor" id="preparation"></a>

In [ ]:
import spikeinterface.full as si
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import os
import csv
import glob
from datetime import datetime
import warnings
warnings.simplefilter("ignore")
%matplotlib widget
print(f"SpikeInterface Version: {si.__version__}")

# 1. Load Recording and Paths <a class="anchor" id="loading"></a>

In [ ]:
# Setting file paths and basic parameters
base_folder = Path('D:/Ephys_C2DRG/')
data_folder = Path("D:/Ephys_C2DRG/2023_9_19/")

recording_paths_list = []
for filename in os.listdir(data_folder):
    if filename.startswith('RawEphysData') and filename.endswith('.bin'):
        recording_paths_list.append(data_folder / filename)
print('Recording Files List:')
print(recording_paths_list)

# parameters associated to the recording in bin format
num_channels = 64 #must know apriori; modify in probe below accordingly.
fs = 30000
gain_to_uV = 0.195
offset_to_uV = 0
rec_dtype = "float32"
time_axis = 0     
time_format = "%H:%M:%S.%f"
n_cpus = os.cpu_count()
n_jobs = n_cpus #n_jobs = -1 :equal to the number of cores.
job_kwargs = dict(n_jobs=n_jobs, chunk_duration="1s", progress_bar=True)

# 2. Concatenate Recording, Filtering and Probe setting

In [ ]:
#Extract and append recording segments to Baserecording object
recordings_list = []
rec = si.read_binary(recording_paths_list, num_chan=num_channels,sampling_frequency=fs,
                           dtype=rec_dtype, gain_to_uV=gain_to_uV, offset_to_uV=offset_to_uV, 
                           time_axis=time_axis, is_filtered=False)
recordings_list.append(rec)#Appends all extracted rec to a list. Kilosort does not support segments. Use concatenation.
recording = si.concatenate_recordings(recordings_list)#Creates Object ConcatenateSegmentRecording
#Filtering recording
recording_f = si.bandpass_filter(recording, freq_min=400, freq_max=5000)
recording_f_cmr = si.common_reference(recording_f, reference='global', operator='median')

In [ ]:
#Get probe from library and set channel mapping
import probeinterface as pi
from probeinterface.plotting import plot_probe
print(f"ProbeInterface version: {pi.__version__}")
manufacturer = 'cambridgeneurotech'
probe_name = 'ASSY-158-H10' #probe_name = 'ASSY-158-F' #probe_name = 'ASSY-158-H6'
probeH10 = pi.get_probe(manufacturer, probe_name)#library: comes with contact_ids and shank_ids info.

#Mapping Intan (device) channels and setting them to probe (RHD-2132/2164)
device_channel_indices = [24,23,25,22,26,21,27,20,28,19,29,18,30,17,31,16,0,15,1,14,2,13,3,12,4,11,5,10,6,9,7,8,
    56,55,57,54,58,53,59,52,60,51,61,50,62,49,63,48,32,47,33,46,34,45,35,44,36,43,37,42,38,41,39,40] #Modify accordingly.
#   88,87,89,86,90,85,91,84,92,83,93,82,94,81,95,80,64,79,65,78,66,77,67,76,68,75,69,74,70,73,71,72,
#   120,119,121,118,122,117,123,116,124,115,125,114,126,113,127,112,96,111,97,110,98,109,99,108,100,107,101,106,102,105,103,104]
probeH10.set_device_channel_indices(device_channel_indices)

#Set and group by shank probe before sorting
recording_f_cmr_prb = recording_f_cmr.set_probe(probeH10, group_mode="by_shank")

#Plotting probe
fig, ax = plt.subplots(figsize=(6, 10))
plot_probe(probeH10, ax=ax, with_contact_id=True, with_device_index=True)
ax.set_xlim(-20, 210)
ax.set_ylim(-75, 320)

# 3. Timestamps, TTLs and Events 
### Converted to seconds and saved

In [ ]:
# Function to calculate time difference in seconds
def time_difference_in_seconds(start_time, end_time):
    start = datetime.combine(datetime.min, start_time)
    end = datetime.combine(datetime.min, end_time)
    return (end - start).total_seconds()

# Load the Timestamps CSV files, extract start time
timestamp_start_df = pd.read_csv(data_folder / 'TimestampsEphys_0.csv', nrows=1, header=None, names=['Timestamps'])
timestamp_start = pd.to_datetime(timestamp_start_df['Timestamps'][0]).time() # Convert timestamps to datetime objects (only time part)

#Load times from timestamps csv files and calculate start time in seconds.
tms_files = sorted(glob.glob(os.path.join(data_folder, "Timestamps*.csv")))
concatenated_segment_times = pd.DataFrame()
for tms_file in tms_files:
    tms_df = pd.read_csv(tms_file, header=None)#, names=['Start_Times', 'End_Times'])#(usecols=[0], nrows=1)
    segment_times = pd.DataFrame({'Start_Times': tms_df.iloc[0], 'End_Times': tms_df.iloc[-1]})
    concatenated_segment_times = pd.concat([concatenated_segment_times, segment_times], ignore_index=True)
concatenated_segment_times['Start_Times'] = pd.to_datetime(concatenated_segment_times['Start_Times']).dt.time # Convert the tms timestamps to timedelta (ignoring date)
concatenated_segment_times['End_Times'] = pd.to_datetime(concatenated_segment_times['End_Times']).dt.time # Convert the tms timestamps to timedelta (ignoring date)
#concatenated_segment_times['Segment_Times'] = pd.to_datetime(concatenated_segment_times['Segment_Times']).dt.time # Convert the tms timestamps to timedelta (ignoring date)
# Apply the time difference function to each row in the tms DataFrame
concatenated_segment_times['Start_Times_seconds'] = concatenated_segment_times['Start_Times'].apply(lambda x: time_difference_in_seconds(timestamp_start, x))
concatenated_segment_times['End_Times_seconds'] = concatenated_segment_times['End_Times'].apply(lambda x: time_difference_in_seconds(timestamp_start, x))
concatenated_segment_times['Segment_duration_seconds'] = concatenated_segment_times['End_Times_seconds'] - concatenated_segment_times['Start_Times_seconds']
#concatenated_segment_times.to_csv(data_folder / 'concatenated_segment_times.csv', mode='x', index=False)
#print(concatenated_segment_times)

In [ ]:
#Load ttl times from csv files, calculate time in seconds
ttl_files = sorted(glob.glob(os.path.join(data_folder, "TTL*.csv")))
concatenated_ttl_times = pd.DataFrame()
for ttl_file in ttl_files:
    TTL_df = pd.read_csv(ttl_file, header=None, names=['TTL_Times'])#(usecols=[0], nrows=1)
    concatenated_ttl_times = pd.concat([concatenated_ttl_times, TTL_df], ignore_index=True)
concatenated_ttl_times['TTL_Times'] = pd.to_datetime(concatenated_ttl_times['TTL_Times']).dt.time # Convert the TTL timestamps to timedelta (ignoring date)
# Apply the time difference function to each row in the TTL DataFrame
concatenated_ttl_times['time_diff_seconds'] = concatenated_ttl_times['TTL_Times'].apply(lambda x: time_difference_in_seconds(timestamp_start, x))
#concatenated_ttl_times.to_csv(data_folder / 'concatenated_ttl_times.csv', mode='x', index=False)
#print(concatenated_ttl_times)

#Load events times from csv files, calculate time in seconds
events_files = sorted(glob.glob(os.path.join(data_folder, "Events*.csv")))
concatenated_event_times = pd.DataFrame()
for event_file in events_files:
    Events_df = pd.read_csv(event_file, header=None, usecols=[0, 1], names=['Stim_start', 'Stim_end'])#(usecols=[0], nrows=1)
    concatenated_event_times = pd.concat([concatenated_event_times, Events_df], ignore_index=True)
concatenated_event_times['Stim_start'] = pd.to_datetime(concatenated_event_times['Stim_start']).dt.time # Convert the TTL timestamps to timedelta (ignoring date)
concatenated_event_times['Stim_end'] = pd.to_datetime(concatenated_event_times['Stim_end']).dt.time # Convert the TTL timestamps to timedelta (ignoring date)
# Apply the time difference function to each row in the TTL DataFrame
concatenated_event_times['time_diff_start_seconds'] = concatenated_event_times['Stim_start'].apply(lambda x: time_difference_in_seconds(timestamp_start, x))
concatenated_event_times['time_diff_end_seconds'] = concatenated_event_times['Stim_end'].apply(lambda x: time_difference_in_seconds(timestamp_start, x))
concatenated_event_times['stim_duration'] = concatenated_event_times['time_diff_end_seconds'] - concatenated_event_times['time_diff_start_seconds']
#concatenated_event_times.to_csv(data_folder / 'concatenated_event_times.csv', mode='x', index=False) 
#print(concatenated_event_times)

# 4. Artifact Removal
### Use Silence Periods Function

In [ ]:
# Plot a single channel trace to store the x coordinates on key_press (for remove artifact function)
coordinates_x_end = [] #holds the end timestamp of the short artifact (< 10ms)
coordinates_x_long = [] #holds the start and end timestamp of a long artifact (> 10ms). Usually ~1 sec
start_time = 5198 * fs
end_time = 5199 * fs
fig, ax = plt.subplots()
trace = recording_f_cmr_prb.get_traces(start_frame=start_time, end_frame=end_time, channel_ids=[8], return_scaled=True)
time_axis = np.arange(start_time, end_time) / fs
ax.plot(time_axis, trace)

# Initialize a variable to keep track of the column position (0 or 1)
click_count = 0
# Function to capture click events
def onkey(event):
    global click_count
    if event.key == 'z': # Only respond to the "z" key
        if event.xdata is not None:
            coordinates_x_end.append((event.xdata))# Store the key_press coordinates in the list
            print(f"Key 'z' pressed at: x={event.xdata}")# Display the coordinates
                
    elif event.key == 'w': # Only respond to the "a" key
        if event.xdata is not None:
            if click_count % 2 == 0:
                # Start a new row for each pair of clicks
                coordinates_x_long.append([event.xdata, None])  # Initialize the second column as None
            else:
                # Update the second column for the latest row
                coordinates_x_long[-1][1] = event.xdata
            print(f"Key 'w' pressed at: x={event.xdata}")
            click_count += 1

# Connect the button press event to the figure
cid = fig.canvas.mpl_connect('key_press_event', onkey)
plt.show()

In [ ]:
# Create arrays with times in seconds for artifact removal. Remove 10 ms for TTL artifacts. 
filename = 'artifacts_coordinates_x.csv'
sec = 0.01 # 10ms
arr = np.array([coordinates_x_end][0], dtype=np.float32)
new_col = arr - sec
coordinates_end = np.column_stack((new_col, arr))
coordinates_long = np.array([coordinates_x_long][0], dtype=np.float32)
artifacts_coordinates_x = np.row_stack((coordinates_end, coordinates_long))
# Create new csv file with values (6 decimal points), and append new values to existing csv file
with open(data_folder / filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for row in artifacts_coordinates_x:
                writer.writerow([f"{value:.6f}" for value in row])

In [ ]:
# Create list of tuples of triggers in frames from artifacts_coordinates_x csv file
filename = 'artifacts_coordinates_x.csv'
triggers_in_frames = []
with open(data_folder / filename, 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
         triggers_in_frames.append([((float(row[0])*fs)), ((float(row[1])*fs))])
triggers_in_frames.sort()
print(triggers_in_frames)

In [ ]:
#Clean recording with silence_periods. List periods is one list per segment of tuples (start_frame, end_frame).
recording_f_cmr_prb_clean = si.silence_periods(recording_f_cmr_prb, list_periods=triggers_in_frames, seed=0, mode='zeros')

In [ ]:
recording_layers1 = dict(common=recording_f_cmr, 
                        clean=recording_f_cmr_prb_clean)

In [ ]:
w = si.plot_traces(recording_layers1, time_range=[6000, 6670], channel_ids=[8],
                return_scaled=True, show_channel_ids=True, backend="ipywidgets")

# 5. Motion correction, followed by Whiten
https://spikeinterface.readthedocs.io/en/stable/how_to/handle_drift.html

In [ ]:
preset_params_ks = si.get_motion_parameters_preset("kilosort_like")
preset = "kilosort_like"
preset_params_ks = ({'direction': 'y', 'rigid': False, 'win_shape': 'rect', 'win_step_um': 100.0, 'win_scale_um': 200.0, 'win_margin_um': None, 
                     'bin_um': 10.0, 'hist_margin_um': 0, 'bin_s': 2.0, 'num_amp_bins': 20, 'num_shifts_global': 15, 'num_iterations': 10, 'num_shifts_block': 5, 
                     'smoothing_sigma': 0.5, 'kriging_sigma': 1, 'kriging_p': 2, 'kriging_d': 2, 'method': 'iterative_template'})

In [ ]:
print("Computing with", preset)
folder = data_folder / "motion_folder_dataset" / preset
recording_corrected, motion, motion_info = si.correct_motion(recording_f_cmr_prb_clean, preset=preset, folder=folder, output_motion=True, 
                                                                 output_motion_info=True, estimate_motion_kwargs=preset_params_ks, **job_kwargs)

In [ ]:
folder = data_folder / "motion_folder_dataset" / preset
motion_info = si.load_motion_info(folder)
# plot motion
fig = plt.figure(figsize=(14, 8))
si.plot_motion_info(motion_info, recording_f_cmr_prb_clean, figure=fig, depth_lim=(-50, 300), 
                    color_amplitude=True, amplitude_cmap="inferno", scatter_decimate=10)
fig.suptitle(f"{preset=}")

In [ ]:
from spikeinterface.sortingcomponents.motion import correct_motion_on_peaks

folder = data_folder / "motion_folder_dataset" / preset
motion_info = si.load_motion_info(folder)
motion = motion_info["motion"]

fig, axs = plt.subplots(ncols=2, figsize=(8, 6), sharey=True)
ax = axs[0]
si.plot_probe_map(recording_f_cmr_prb_clean, ax=ax)
peaks = motion_info["peaks"]
sr = rec.get_sampling_frequency()
time_lim0 = 0.0
time_lim1 = 3000.0
mask = (peaks["sample_index"] > int(sr * time_lim0)) & (peaks["sample_index"] < int(sr * time_lim1))
sl = slice(None, None, 5)
amps = np.abs(peaks["amplitude"][mask][sl])
amps /= np.quantile(amps, 0.95)
c = plt.get_cmap("inferno")(amps)

color_kargs = dict(alpha=0.5, s=2, c=c)

peak_locations = motion_info["peak_locations"]
ax.scatter(peak_locations["x"][mask][sl], peak_locations["y"][mask][sl], **color_kargs)
ax.set_ylim(-75, 250)
ax.set_xlim(-45, 80)

peak_locations2 = correct_motion_on_peaks(peaks, peak_locations, motion,rec)
ax = axs[1]
si.plot_probe_map(recording_f_cmr_prb_clean, ax=ax)
ax.scatter(peak_locations2["x"][mask][sl], peak_locations2["y"][mask][sl], **color_kargs)
ax.set_ylim(-75, 320)
ax.set_xlim(-45, 80)
fig.suptitle(f"{preset=}")

In [ ]:
recording_f_cmr_prb_clean_corr_w = si.whiten(recording_corrected, int_scale=200)

recording_layers = dict(common=recording_f_cmr, 
                        clean=recording_f_cmr_prb_clean,
                        whiten=recording_f_cmr_prb_clean_corr_w)

In [ ]:
w = si.plot_traces(recording_layers, time_range=[350, 360], channel_ids=[8, 45],
                return_scaled=True, show_channel_ids=True, backend="ipywidgets")

# 6. Saving filtered, clean, motion corrected and whitened recording (with probe) to binary.

In [ ]:
recording_f_cmr_prb_clean_corr_w.save(format='binary', folder=data_folder / "recording_preprocessed", overwrite=True, **job_kwargs)